In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import matplotlib

In [ ]:
IMAGE_SIZE = 28
BATCH_SIZE = 50
NUM_ITERATIONS = 3000
CONVOLUTION_DEPTH = 32
LEARNING_RATE = 0.0002

In [ ]:
data = pd.read_csv('train.csv')
data = data.drop('label', axis=1)
data = data.as_matrix() / 255.0

In [ ]:
# helper functions
def display_image(image):
    image = image.reshape(IMAGE_SIZE, IMAGE_SIZE)
    plot.axis('off')
    plot.imshow(image, cmap=matplotlib.cm.binary)
    plot.show()
    
    
def get_sample_z(size=(1, 100)):
    return np.random.normal(size=size)

# Loading next batch
epochs_completed = 0
index_in_epoch = 0
num_examples = data.shape[0]


def next_batch(batch_size):
    global data
    global index_in_epoch
    global epochs_completed
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        # finished epoch
        epochs_completed += 1
        # shuffle the data
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        data = data[perm]
        # start next epoch
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    return data[start:end]

In [ ]:
def generator(z, batch_size, reuse=False):
    with tf.variable_scope('generator', reuse=reuse):
        h0 = tf.layers.dense(inputs=z,
                             units=CONVOLUTION_DEPTH * 2 * 2 * 8)
        h0 = tf.reshape(h0, [batch_size, 2, 2, CONVOLUTION_DEPTH * 8])
        h0 = tf.contrib.layers.batch_norm(h0,
                                          decay=0.9,
                                          scale=True,
                                          is_training=True,
                                          epsilon=1e-5)
        h0 = tf.nn.relu(h0)

        g_w1 = tf.get_variable('g_w1', [5, 5, CONVOLUTION_DEPTH * 4 , CONVOLUTION_DEPTH * 8],
                               dtype=tf.float32,
                               initializer=tf.random_normal_initializer(stddev=0.02))
        g_b1 = tf.get_variable('g_b1', [CONVOLUTION_DEPTH * 4],
                               initializer=tf.random_normal_initializer(stddev=0.02))
        h1 = tf.nn.conv2d_transpose(h0,
                                    g_w1,
                                    strides=[1, 2, 2, 1],
                                    output_shape=[batch_size, 4, 4, CONVOLUTION_DEPTH * 4])
        h1 = h1 + g_b1
        h1 = tf.contrib.layers.batch_norm(h1,
                                          decay=0.9,
                                          scale=True,
                                          is_training=True,
                                          epsilon=1e-5)
        h1 = tf.nn.relu(h1)

        g_w2 = tf.get_variable('g_w2',
                               [5, 5, CONVOLUTION_DEPTH * 2, CONVOLUTION_DEPTH * 4],
                               dtype=tf.float32,
                               initializer=tf.random_normal_initializer(stddev=0.02))
        g_b2 = tf.get_variable('g_b2', [CONVOLUTION_DEPTH * 2],
                               initializer=tf.random_normal_initializer(stddev=0.02))
        h2 = tf.nn.conv2d_transpose(h1,
                                    g_w2,
                                    output_shape=[batch_size, 7, 7, CONVOLUTION_DEPTH * 2],
                                    strides=[1, 2, 2, 1])
        h2 = h2 + g_b2

        h2 = tf.contrib.layers.batch_norm(h2,
                                          decay=0.9,
                                          scale=True,
                                          is_training=True,
                                          epsilon=1e-5)
        h2 = tf.nn.relu(h2)

        g_w3 = tf.get_variable('g_w3',
                               [5, 5, CONVOLUTION_DEPTH * 1, CONVOLUTION_DEPTH * 2],
                               dtype=tf.float32,
                               initializer=tf.random_normal_initializer(stddev=0.02))
        g_b3 = tf.get_variable('g_b3', [CONVOLUTION_DEPTH * 1],
                               initializer=tf.random_normal_initializer(stddev=0.02))
        h3 = tf.nn.conv2d_transpose(h2,
                                    g_w3,
                                    output_shape=[batch_size, 14, 14, CONVOLUTION_DEPTH * 1],
                                    strides=[1, 2, 2, 1])
        h3 = h3 + g_b3

        h3 = tf.contrib.layers.batch_norm(h3,
                                          decay=0.9,
                                          scale=True,
                                          is_training=True,
                                          epsilon=1e-5)
        h3 = tf.nn.relu(h3)

        g_w4 = tf.get_variable('g_w4',
                               [5, 5, 1, CONVOLUTION_DEPTH * 1],
                               dtype=tf.float32,
                               initializer=tf.random_normal_initializer(stddev=0.02))
        g_b4 = tf.get_variable('g_b4', [1],
                               initializer=tf.random_normal_initializer(stddev=0.02))
        h4 = tf.nn.conv2d_transpose(h3,
                                    g_w4,
                                    output_shape=[batch_size, 28, 28, 1],
                                    strides=[1, 2, 2, 1])
        h4 = h4 + g_b4

        return tf.nn.tanh(h4)

In [ ]:
def discriminator(image, reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        image = tf.reshape(image, [-1, 28, 28, 1])
        h0 = tf.layers.conv2d(inputs=image,
                              kernel_size=[5, 5],
                              filters=CONVOLUTION_DEPTH,
                              strides=(2, 2),
                              padding='SAME')
        h0 = tf.maximum(h0, 0.2*h0)

        h1 = tf.layers.conv2d(inputs=h0,
                              kernel_size=[5, 5],
                              filters=2*CONVOLUTION_DEPTH,
                              strides=(2, 2),
                              padding='SAME')
        h1 = tf.contrib.layers.batch_norm(h1,
                                          decay=0.9,
                                          scale=True,
                                          is_training=True,
                                          epsilon=1e-5)
        h1 = tf.maximum(h1, 0.2*h1)

        h2 = tf.layers.conv2d(inputs=h1,
                              kernel_size=[5, 5],
                              filters=4 * CONVOLUTION_DEPTH,
                              strides=(2, 2),
                              padding='SAME')
        h2 = tf.contrib.layers.batch_norm(h2,
                                          decay=0.9,
                                          scale=True,
                                          is_training=True,
                                          epsilon=1e-5)
        h2 = tf.maximum(h2, 0.2 * h2)

        h3 = tf.layers.conv2d(inputs=h2,
                              kernel_size=[5, 5],
                              filters=8 * CONVOLUTION_DEPTH,
                              strides=(2, 2),
                              padding='SAME')
        h3 = tf.contrib.layers.batch_norm(h3,
                                          decay=0.9,
                                          scale=True,
                                          is_training=True,
                                          epsilon=1e-5)
        h3 = tf.maximum(h3, 0.2 * h3)

        h4 = tf.layers.dense(inputs=h3,
                             units=1)

        return tf.nn.sigmoid(h4), h4

In [ ]:
Z_in = tf.placeholder(tf.float32, shape=[None, 100])
image_in = tf.placeholder(tf.float32, shape=[None,
                                             IMAGE_SIZE * IMAGE_SIZE])

gen_sample = generator(Z_in, BATCH_SIZE)

discriminator_data, data_logits = discriminator(image_in)
discriminator_model, model_logits = discriminator(gen_sample,
                                                  reuse=True)

In [ ]:
# losses
discriminator_loss_data = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=data_logits,
                                                                                  labels=tf.ones_like(discriminator_data)))

discriminator_loss_model = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=model_logits,
                                                                                   labels=tf.zeros_like(discriminator_model)))

discriminator_loss = discriminator_loss_data + discriminator_loss_model

generator_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=model_logits,
                                                                        labels=tf.ones_like(discriminator_model)))

In [ ]:
all_vars = tf.trainable_variables()
generator_vars = [var for var in all_vars if var.name.startswith('generator')]
discriminator_vars = [var for var in all_vars if var.name.startswith('discriminator')]

discriminator_optimize = tf.train.AdamOptimizer(LEARNING_RATE).minimize(discriminator_loss,
                                                                        var_list=discriminator_vars)
generator_optimize = tf.train.AdamOptimizer(LEARNING_RATE).minimize(generator_loss,
                                                                    var_list=generator_vars)

In [ ]:
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()

sess.run(init)

In [ ]:
for i in range(NUM_ITERATIONS):
    image_batch = next_batch(BATCH_SIZE)
    sess.run(discriminator_optimize, feed_dict={Z_in: get_sample_z([BATCH_SIZE, 100]), image_in: image_batch})
    sess.run(generator_optimize, feed_dict={Z_in: get_sample_z([BATCH_SIZE, 100])})
    sess.run(generator_optimize, feed_dict={Z_in: get_sample_z([BATCH_SIZE, 100])})
    
    disc_losses = sess.run(discriminator_loss, feed_dict={Z_in: get_sample_z([BATCH_SIZE, 100]),
                                                          image_in: image_batch})
    gen_losses = sess.run(generator_loss, feed_dict={Z_in: get_sample_z([BATCH_SIZE, 100])})
    
    if i % 100 == 0:
        print('Step {} => Discriminator: {} | Generator: {}'.format(i, disc_losses, gen_losses))

In [ ]:
sample_g = generator(Z_in, 1, True)
x = sess.run(sample_g, feed_dict={Z_in: get_sample_z([1, 100])})
display_image(x)